In [1]:
import itertools
import dlib
import enum
import numpy as np
import time
import cv2
import imutils
from imutils.video import FPS
import pandas as pd
from scipy.spatial import distance as dist
from collections import OrderedDict

In [2]:
class Direction(enum.Enum):
  ONBOARD = 1
  OFFBOARD= -1
  NAN = 0

### Centroid Tracker class

In [3]:
class CentroidTracker:
	def __init__(self, maxDisappeared=50, maxDistance=50):
		# initialize the next unique object ID along with two ordered
		# dictionaries used to keep track of mapping a given object
		# ID to its centroid and number of consecutive frames it has
		# been marked as "disappeared", respectively
		self.nextObjectID = 0
		self.objects = OrderedDict()
		self.disappeared = OrderedDict()

		# store the number of maximum consecutive frames a given
		# object is allowed to be marked as "disappeared" until we
		# need to deregister the object from tracking
		self.maxDisappeared = maxDisappeared

		# store the maximum distance between centroids to associate
		# an object -- if the distance is larger than this maximum
		# distance we'll start to mark the object as "disappeared"
		self.maxDistance = maxDistance

	def register(self, centroid):
		# when registering an object we use the next available object
		# ID to store the centroid
		self.objects[self.nextObjectID] = centroid
		self.disappeared[self.nextObjectID] = 0
		self.nextObjectID += 1

	def deregister(self, objectID):
		# to deregister an object ID we delete the object ID from
		# both of our respective dictionaries
		print("deregistered {}-Object".format(objectID))
		to = trackableObjects.get(objectID, None)
		to.end_of_tracking()
		del self.objects[objectID]
		del self.disappeared[objectID]

	def update(self, rects):
		# check to see if the list of input bounding box rectangles
		# is empty
		if len(rects) == 0:
			# loop over any existing tracked objects and mark them
			# as disappeared
			for objectID in list(self.disappeared.keys()):
				self.disappeared[objectID] += 1

				# if we have reached a maximum number of consecutive
				# frames where a given object has been marked as
				# missing, deregister it
				if self.disappeared[objectID] > self.maxDisappeared:
					self.deregister(objectID)

			# return early as there are no centroids or tracking info
			# to update
			return self.objects

		# initialize an array of input centroids for the current frame
		inputCentroids = np.zeros((len(rects), 2), dtype="int")

		# loop over the bounding box rectangles
		for (i, (startX, startY, endX, endY)) in enumerate(rects):
			# use the bounding box coordinates to derive the centroid
			cX = int((startX + endX) / 2.0)
			cY = int((startY + endY) / 2.0)
			inputCentroids[i] = (cX, cY)

		# if we are currently not tracking any objects take the input
		# centroids and register each of them
		if len(self.objects) == 0:
			for i in range(0, len(inputCentroids)):
				self.register(inputCentroids[i])

		# otherwise, are are currently tracking objects so we need to
		# try to match the input centroids to existing object
		# centroids
		else:
			# grab the set of object IDs and corresponding centroids
			objectIDs = list(self.objects.keys())
			objectCentroids = list(self.objects.values())

			# compute the distance between each pair of object
			# centroids and input centroids, respectively -- our
			# goal will be to match an input centroid to an existing
			# object centroid
			D = dist.cdist(np.array(objectCentroids), inputCentroids)

			# in order to perform this matching we must (1) find the
			# smallest value in each row and then (2) sort the row
			# indexes based on their minimum values so that the row
			# with the smallest value as at the *front* of the index
			# list
			rows = D.min(axis=1).argsort()

			# next, we perform a similar process on the columns by
			# finding the smallest value in each column and then
			# sorting using the previously computed row index list
			cols = D.argmin(axis=1)[rows]

			# in order to determine if we need to update, register,
			# or deregister an object we need to keep track of which
			# of the rows and column indexes we have already examined
			usedRows = set()
			usedCols = set()

			# loop over the combination of the (row, column) index
			# tuples
			for (row, col) in zip(rows, cols):
				# if we have already examined either the row or
				# column value before, ignore it
				if row in usedRows or col in usedCols:
					continue

				# if the distance between centroids is greater than
				# the maximum distance, do not associate the two
				# centroids to the same object
				if D[row, col] > self.maxDistance:
					continue

				# otherwise, grab the object ID for the current row,
				# set its new centroid, and reset the disappeared
				# counter
				objectID = objectIDs[row]
				self.objects[objectID] = inputCentroids[col]
				self.disappeared[objectID] = 0

				# indicate that we have examined each of the row and
				# column indexes, respectively
				usedRows.add(row)
				usedCols.add(col)

			# compute both the row and column index we have NOT yet
			# examined
			unusedRows = set(range(0, D.shape[0])).difference(usedRows)
			unusedCols = set(range(0, D.shape[1])).difference(usedCols)

			# in the event that the number of object centroids is
			# equal or greater than the number of input centroids
			# we need to check and see if some of these objects have
			# potentially disappeared
			if D.shape[0] >= D.shape[1]:
				# loop over the unused row indexes
				for row in unusedRows:
					# grab the object ID for the corresponding row
					# index and increment the disappeared counter
					objectID = objectIDs[row]
					self.disappeared[objectID] += 1

					# check to see if the number of consecutive
					# frames the object has been marked "disappeared"
					# for warrants deregistering the object
					if self.disappeared[objectID] > self.maxDisappeared:
						self.deregister(objectID)

			# otherwise, if the number of input centroids is greater
			# than the number of existing object centroids we need to
			# register each new input centroid as a trackable object
			else:
				for col in unusedCols:
					self.register(inputCentroids[col])

		# return the set of trackable objects
		return self.objects

### Tracked object data class, makes for unique tracking

In [4]:
class TrackableObject:
	def __init__(self, objectID, centroid, first_frame, label, direction, fisheries_data):
		# store the object ID, then initialize a list of centroids
		# using the current centroid
		self.objectID = objectID
		self.centroids = [centroid]
		self.first_frame = first_frame
		self.label = label
		self.direction = direction

		self.fishery_export = fisheries_data

		# initialize a boolean used to indicate if the object has
		# already been counted or not
		self.counted = False

	def end_of_tracking(self):
	  # Add data from tracking to export
		self.fishery_export.addData(self.label,
		                            self.first_frame, 
																self.last_frame, 
		                       			"Onboard" if self.direction == Direction.ONBOARD 
													        else "Offboard")

### John's data export class FisheriesData

In [5]:
import pandas as pd

class FisheriesData:
    def __init__(self):
        self.species = "species"
        self.timeStampStart = "time_stamp_start"
        self.timeStampEnd = "time_stamp_end"
        self.direction = "direction"
        self.dataFrame = []
        self.speciesList = []
        self.timeStampStartList = []
        self.timeStampEndList = []
        self.directionList = []
    
    def makeDF(self):
        self.dataFrame = pd.DataFrame({self.species:self.speciesList,self.timeStampStart:self.timeStampStartList,self.timeStampEnd:self.timeStampEndList,self.direction:self.directionList})
    
    def addData(self, species, timeStampStart, timeStampEnd, direction):
        self.speciesList.append(species)
        self.timeStampStartList.append(timeStampStart)
        self.timeStampEndList.append(timeStampEnd)
        self.directionList.append(direction)

    def addDictData(self, dictionary):
        for k, v in dictionary.items():
            for kk, vv in v.items():
                if type(kk) is tuple:
                    x, y = kk
                    self.timeStampStartList.append(x)
                    self.timeStampEndList.append(y)
                    self.speciesList.append(vv)
                else:
                    self.directionList.append(vv)

    def writeCSV(self, fileName):
        file_name = fileName + '.csv'
        self.dataFrame.to_csv(file_name, sep='\t', encoding='utf-8')

    def writeExcel(self, fileName):
        file_name = fileName + '.xlsx'
        self.dataFrame.to_excel(file_name, sheet_name='sheet1', index=False)

    def writeXML(self, fileName):
        file_name = fileName + '.xml'
        self.dataFrame.to_xml(file_name)

    def writeJSON(self, fileName):
        file_name = fileName + '.json'
        self.dataFrame.to_json(file_name, orient='records', indent=2)

### Bounding Box Class (tracker alt)

In [6]:
class BBox:
  x1 = 0
  y1 = 0
  x2 = 0
  y2 = 0
  label = ""
  
  # Consturctor for BBox taking x1,x2,y1,y2
  # def __init__(self, x1, x2, y1, y2, label):
  #   if x1>x2 or y1>y2:
  #     raise ValueError("Coordinates are invalid")
  #   if label == "" or not label:
  #     raise ValueError("Please include label")
  #   self.x1, self.y1, self.x2, self.y2 = x1, y1, x2, y2
  #   self.label = label

  # Constructor for BBox taking x,y w,h
  # Use for Yolo detections
  def __init__(self, x:int, y:int, w:int, h:int, label:str):
    if x>(x+w) or y>(y+h):
      raise ValueError("Coordinates are invalid")
    if label == "" or not label:
      raise ValueError("Please include label")
    self.x1, self.y1, self.x2, self.y2 = x, y, (x+w), (y+h)
    self.label = label

  # Takes a BBox class instance to compare the overlap area 
  # and returns an value for overlap area
  def intersection_area(self, bbox: object) -> float:
    if type(bbox) != self:
      raise TypeError("BBox should be an instance of a BBox Class")
    dx = min(self.x1, bbox.x1) - max(self.x2, bbox.x2)
    dy = min(self.y1, bbox.y1) - max(self.y2, bbox.y2)
    if (dx>=0) and (dy>=0):
      if self.label == bbox.label:
        return dx*dy
    return -1

  # Takes a BBox and compares x positions 
  def direction_of_motion(self, bbox: object):
    if type(bbox) != self:
      raise TypeError("BBox should be an instance of a BBox Class")
    dx = self.x1 - bbox.x1
    if dx > 0:
      return Direction.ONBOARD
    elif dx < 0:
      return Direction.OFFBOARD
    elif dx == 0:
      return Direction.NAN

### Black and white the frame

In [7]:
def get_binary_frame(frame, frame2):
  # Convert frame to RGB frame
  rgb_frame = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2RGB)
  rgb_frame2 = cv2.cvtColor(src=frame2, code=cv2.COLOR_BGR2RGB)

  # Convert the image to grayscale format
  gray_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2GRAY)
  gray_frame2 = cv2.cvtColor(rgb_frame2, cv2.COLOR_BGR2GRAY)

  # Blur the image for smoothing
  prepared_frame = cv2.GaussianBlur(src=gray_frame, ksize=(5, 5), sigmaX=0)
  previous_frame = cv2.GaussianBlur(src=gray_frame2, ksize=(5, 5), sigmaX=0)

  # Calculate difference and update previous frame
  diff_frame = cv2.absdiff(src1=previous_frame, src2=prepared_frame)

  # Dilute the image a bit to make differences more seeable; more suitable for contour detection
  kernel = np.ones((5, 5))
  diff_frame = cv2.dilate(diff_frame, kernel, 1)

  # Only take different areas that are different enough (>20 / 255)
  thresh_frame = cv2.threshold(src=diff_frame, thresh=99, maxval=250, type=cv2.THRESH_BINARY)[1]
  return thresh_frame

# Yolo Detector modified from object_detector.ipynb

In [8]:
# INPUT_FILE='penguin.mp4'

In [9]:
INPUT_FILE='Fish_Training_3.mp4'

In [10]:
# INPUT_FILE='crab.mp4'

In [11]:
OUTPUT_FILE='output2.mp4'

H=None
W=None

# capture input video
video_capture = cv2.VideoCapture(INPUT_FILE)

# get input video's frame size
frame_width = int(video_capture.get(3))
frame_height = int(video_capture.get(4))
frame_size = (frame_width,frame_height)

# get input video's fps
input_fps = video_capture.get(cv2.CAP_PROP_FPS)

fps = FPS().start()

# fourcc = cv2.VideoWriter_fourcc(*"MJPG") # for avi
fourcc = cv2.VideoWriter_fourcc(*"mp4v") # for mp4
writer = cv2.VideoWriter(OUTPUT_FILE, fourcc, input_fps, frame_size, True)

cnt = 0

trackers = []
ct = CentroidTracker(maxDisappeared=10, maxDistance=55)
trackableObjects = {}

# initialize FisheriesData to use for aggregating export data
fishery_export = FisheriesData()
last_frame = None
# iterate through video frames
while True:
	rects = []
	cnt+=1

	# print ("Frame number", cnt)
	ok, image = video_capture.read()
	if last_frame is None:
		last_frame = image
		
	# blob = get_binary_frame(image, last_frame)
	# blob = cv2.dnn.blobFromImage(image, 1)
	for tracker in trackers:
		# update the tracker and grab the updated position
		tracker.update(image)
		pos = tracker.get_position()

		# unpack the position object
		startX = int(pos.left())
		startY = int(pos.top())
		endX = int(pos.right())
		endY = int(pos.bottom())

		# add the bounding box coordinates to the rectangles list
		rects.append((startX, startY, endX, endY))

	key = cv2.waitKey(0) & 0xFF

	if key == ord("q"):
		break
	elif key == ord("a"):
		box = cv2.selectROI(image)
		(x, y, width, height) = box
		# construct a dlib rectangle object from the bounding
		# box coordinates and then start the dlib correlation
		# tracker
		tracker = dlib.correlation_tracker()
		rect = dlib.rectangle(x, y, x+width, y+height)
		tracker.start_track(image, rect)

		# add the tracker to our list of trackers so we can
		# utilize it during skip frames
		trackers.append(tracker)

		# update our list of bounding box coordinates, confidences,
		# and class IDs
		# boxes.append([x, y, int(width), int(height)])
	elif key == ord("f"):
		rects = []
	elif key == ord("r"):
		rects = []
		trackers = []
		trackableObjects = {}

	# Update the centroid tracker to associate the old object centroid
	# with the newly computed centroid
	objects = ct.update(rects)
	# print(objects)
	# loop over the tracked objects
	for (objectID, centroid) in objects.items():
		# check to see if a trackable object exists for the current
		# object ID
		to = trackableObjects.get(objectID, None)

		# if there is no existing trackable object, create one
		if to is None:
			#objectID, centroid, first_frame, label, direction, fisheries_data
			to = TrackableObject(objectID=objectID, centroid=centroid, first_frame=cnt, 
			                     label="Object-{}".format(objectID), direction=Direction.ONBOARD, 
													 fisheries_data=fishery_export)
			print("created new trackable object: {}".format("temp_label"))
		# otherwise, there is a trackable object so we can utilize it
		# to determine direction
		else:
			# the difference between the y-coordinate of the *current*
			# centroid and the mean of *previous* centroids will tell
			# us in which direction the object is moving (negative for
			# 'up' and positive for 'down')
			y = [c[1] for c in to.centroids]
			direction = centroid[1] - np.mean(y)
			to.centroids.append(centroid)

			# check to see if the object has been counted or not
			if not to.counted:
				# if the direction is negative (indicating the object
				# is moving up)
				if direction < 0 :
					to.direction = Direction.ONBOARD
					to.counted = True
				# if the direction is positive (indicating the object
				# is moving down)
				elif direction > 0:
					to.direction = Direction.OFFBOARD
					to.counted = True
		
		# print("{}-{}".format(to.objectID, to.label))
		to.last_frame = cnt
		# store the trackable object in our dictionary
		trackableObjects[objectID] = to
		# Display the Centroid in the frame
		text = "ID {}".format(objectID)
		cv2.putText(image, text, (centroid[0] - 10, centroid[1] - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.5, (1, 1, 1), 2)
		cv2.circle(image, (centroid[0], centroid[1]), 4, (255, 255, 255), -1)

	# show the output image
	cv2.imshow("output", image)
	# cv2.imshow("Blob output", blob)
	# Convert image colors back to bgr from rgb for presentation
	# image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
	writer.write(cv2.resize(image,frame_size))
	fps.update()
	
	last_frame = image
	if ok is None:
		break

fps.stop()

print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# Export FisheriesData
fishery_export.makeDF()
fishery_export.writeCSV('Testing')
fishery_export.writeJSON('Testing')
# fishery_export.writeXML('Testing')

# do a bit of cleanup
cv2.destroyAllWindows()

# release the file pointers
print("[INFO] cleaning up...")
writer.release()
video_capture.release()

created new trackable object: temp_label
created new trackable object: temp_label
created new trackable object: temp_label
created new trackable object: temp_label
deregistered 0-Object


AttributeError: 'NoneType' object has no attribute 'end_of_tracking'